In [ ]:
import time
import numpy as np
import pylab
import matplotlib.pyplot as plt
import pandas as pd
import math
from IPython.display import Image

%matplotlib inline

In [ ]:
orders = pd.read_csv('./orders.csv')

In [ ]:
orders.head(13)

In [ ]:
order_product = pd.read_csv('./order_products__prior.csv', index_col='order_id')

In [ ]:
order_product.head(10)

prediction of basket size

In [ ]:
user_len = max(orders['user_id'])

In [ ]:
t = 0
i = 0
bask_len = []
for user_id in range(1, user_len + 1):
    user_orders = orders[orders['user_id'] == user_id]['order_id'][:-1].values
    prod = 0
    user = 0
    for user_order in user_orders:
        
        tmp = order_product.loc[user_order]['product_id']
        y = len(tmp.values) if type(tmp) is pd.core.series.Series else 1
        x = np.array(orders.loc[orders['order_id'] == user_order]
                     [['order_dow', 'order_hour_of_day', 'days_since_prior_order']])[0]
        if user != 0 :
            bask_len.append(np.concatenate((x,[prod/user,y])))
        prod+=y
        user+=1
        i+=1 
#break when we dont have all the time
#     if i > 40000:
#         break
    t+=1
    if t%100==0:
        print(t)  

In [ ]:
bask_len = np.array(bask_len)

In [ ]:
bask_len = bask_len[~np.isnan(bask_len).any(axis=1)]

In [ ]:
data = bask_len[:,[0,1,2,3]]

In [ ]:
labels  = bask_len[:,4]

In [ ]:
test = data[2900000:]
test_labels = labels[2900000:]

In [ ]:
train = data[:2900000]
tr_labels  =labels[:2900000]

In [ ]:
np.var(labels)

In [ ]:
from sklearn.neural_network import MLPRegressor, MLPClassifier

In [ ]:
cls = MLPRegressor(hidden_layer_sizes=(100,100 ))

In [ ]:
cls.fit(train, tr_labels)

In [ ]:
predicted = cls.predict(test)

In [ ]:
np.mean(np.abs(predicted - test_labels))

In [ ]:
predicted, test_labels

In [ ]:
plt.plot(predicted[:100])
plt.plot(test_labels[:100])

In [ ]:
# load mean baskets of users for compare with new model
mean_b = np.load('mean_basket.npy')

In [ ]:
for_predict = []

In [ ]:
t = 0
mean_basket = np.zeros(user_len + 1, dtype=np.int16)

my_orders = orders[orders['eval_set'] != 'prior']
for index,row in my_orders.iterrows():
    user_id = row['user_id']
    x = np.array(list(row)[4:] + [mean_b[user_id]]).reshape(1,4)
    mean_basket[user_id] =cls.predict(x)
    t+=1
    
    if t%1000==0:
        print(t)   

In [ ]:
np.mean(np.abs(mean_basket-mean_b))

Markov chain

In [ ]:
product_len = max(order_product['product_id'])

In [ ]:
# init of transaction matrix
transaction = np.zeros(product_len ** 2, dtype=np.float16).reshape(product_len,product_len)

In [ ]:
num_prod = np.zeros(product_len)

In [ ]:
# markov chains transaction matrix
t = 0

for user in range(1, user_len+1):
    baskets = []
    order_ids = orders_by_users.loc[user_id]['order_id'].values
    
    for order_id in order_ids:
        products = order_product.loc[order_id]['product_id']
        products = np.array(products) if type(products) is pd.core.series.Series else np.array([products])
        baskets.append(products)  

    for ind_bas in range(len(baskets)-1):
        for ind_pr, product in enumerate(baskets[ind_bas]):
            num_prod[product-1] +=1
            for next_product in baskets[ind_bas+1]:
                transaction[product-1][next_product-1] += 1    
    t+=1
    if t%1000==0:
        print(t)    

In [ ]:
for index, prod_transaction in enumerate(transaction):
    transaction[index] = transaction[index]/num_prod[index]


In [ ]:
transaction.shape

In [ ]:
# mean basket of user, for first step we will assume that basket size will be the mean size of all user's basket
t = 0
mean_basket = np.zeros(user_len + 1, dtype=np.int16)
for user_id in range(1, user_len + 1):
    user_orders = orders[orders['user_id'] == user_id]['order_id'][:-1].values
    prods = 0
    for user_order in user_orders:
        tmp = order_product.loc[user_order]['product_id']
        prods += len(tmp.values) if type(tmp) is pd.core.series.Series else 1
    mean_basket[user_id] = prods / len(user_orders)
    t+=1
    if t%1000==0:
        print(t)   

In [ ]:
training_points = orders[orders['eval_set'] == 'test']
ids = training_points.index

In [ ]:
# read each user's products
dmp = np.arange(0, len(transaction) + 1, 1)
user_all_products = {}
for _id in range(user_len + 1):
    user_all_products[str(_id)] = dmp
with open('user_unique_products.csv', 'r') as f:
    for line in f:
        _id, prods = line.split(',')
        user_all_products[_id] = np.array([int(x) for x in prods.split(' ')])

In [ ]:
# predict
%%time
t = 0
with open('submission.csv', 'w') as f:
    f.write('order_id,products\n')
    for _id in ids:
        unique_prods = user_all_products[str(orders.loc[_id]['user_id'])] - 1
        last_basket_products = order_product.loc[orders.loc[_id - 1]['order_id']]['product_id'] - 1
        last_basket_products = np.array(last_basket_products) if type(last_basket_products) is pd.core.series.Series else np.array([last_basket_products])
        
        prob_sum = np.sum(transaction[last_basket_products, :], axis=0)
        cids = np.where(prob_sum > 0)[0]
        cids = np.intersect1d(cids, unique_prods)
        pred = pd.DataFrame(data={'id': cids, 'p': prob_sum[cids]})
        
        mb = mean_basket[orders[orders['order_id'] == _id]['user_id'].values[0]]
        top = pred.sort_values('p')[-mb:]
                
        f.write(str(orders.loc[_id]['order_id']))
        f.write(',')
        if len(cids) > 5000:
            f.write('None')
        else:
            f.write(str(top['id'].values + 1)[1:-1])
        f.write('\n')
        
        t+=1
        if t%1000==0:
            print(t)

In [ ]:
# afterprocessing for submission 
with open('submission.csv', 'r') as f:
    with open('sub_sub.csv', 'w') as g:
        for line in f:
            if ',' in line:
                g.write('\n')
            g.write(line[:-1])
        g.write('\n')